In [1]:
from pynq import Overlay
import pynq.lib.dma
import numpy as np
from pynq import Xlnk

In [2]:
def FeedForwardConv0(bias, weight_3d, image_3d):    
    pad = []
    for i in range(3):
        pad.append(np.zeros((34, 34), dtype=np.int32))
        pad[-1][1:33, 1:33] = np.asarray(image_3d[i], dtype=np.int32)
        pad[-1] = pad[-1].flatten().tolist()

    flat_weight = []
    for i in range(3):
        flat_weight.append([weight for weight_2d in weight_3d[i] for weight in weight_2d])
        
    feed = [bias]
    feed = feed + flat_weight[0] + pad[0]
    feed = feed + flat_weight[1] + pad[1]
    feed = feed + flat_weight[2] + pad[2]
    feed = feed + [0]
    samples = np.asarray(feed).astype(np.int32)
    slen = len(samples)
    
    overlay = Overlay('fp.bit')
    dma = overlay.axi_dma_0
    overlay.reset()
    
    xlnk = Xlnk()
    out_buffer = xlnk.cma_array(shape=(slen+5+1+1,), dtype=np.int32)
    in_buffer = xlnk.cma_array(shape=(slen,), dtype=np.int32)
    np.copyto(in_buffer, samples)
    
    dma.sendchannel.transfer(in_buffer)
    dma.recvchannel.transfer(out_buffer)
    
    out = []
    for i in range(32):
        off_index = 2330 + (i * 34) + 66 + 5 + 9 + 5 + 1 
        out = out + list(out_buffer[off_index : off_index + 32])

    out = np.asarray(out).reshape((32, 32))
    return out

In [3]:
bias = 3
weight = np.ones((3, 3, 3)).tolist()
image = np.asarray(list(range(32*32))*3, dtype=np.int32).reshape((3, 32,32)).tolist()

print(FeedForwardConv0(bias, weight, image))

[[  201   309   327 ...,   813   831   561]
 [  588   894   921 ...,  1650  1677  1128]
 [ 1164  1758  1785 ...,  2514  2541  1704]
 ..., 
 [16716 25086 25113 ..., 25842 25869 17256]
 [17292 25950 25977 ..., 26706 26733 17832]
 [11721 17589 17607 ..., 18093 18111 12081]]
